In [3]:
import h5py
import numpy as np

case_path = r"C:/Users/spearlab05/Desktop/Galerkin ROM/Data/test.cas.h5"
data_path = r"C:/Users/spearlab05/Desktop/Galerkin ROM/Data/test.dat.h5"

# ----------------------------
# 1) CASE: 좌표와 face 연결 읽기
# ----------------------------
with h5py.File(case_path, "r") as f:
    # mesh id 추출 (보통 '1')
    mesh_ids = sorted(k for k in f["meshes"].keys())
    mesh_id = mesh_ids[0]
    base = f[f"meshes/{mesh_id}"]

    # (a) 노드 좌표
    # 보통 coords/3 에 3차원 좌표가 들어있음. 모양은 (N,3) 또는 (3,N)일 수 있음
    coords_ds = base["nodes/coords/3"]
    coords = coords_ds[()]
    if coords.ndim == 2 and coords.shape[0] == 3:
        coords = coords.T  # (3, N) -> (N, 3)
    print("[CASE] coordinates:", coords.shape, "(first 3 rows)\n", coords[:3])

    # (b) face-노드 연결
    # nodes: 모든 face의 노드 ID를 1D로 이어붙인 배열 (Fluent는 보통 1-based 인덱스)
    # nnodes: 각 face가 가진 노드 수 (예: [4,4,3,4,...])
    nodes_flat = base["faces/nodes/1/nodes"][()]
    nnodes = base["faces/nodes/1/nnodes"][()]
    # 1-based일 가능성 높음 → 0-based로 바꾸고 싶다면 -1
    nodes_flat0 = nodes_flat - 1

    # ragged 연결 리스트로 복원
    cuts = np.cumsum(nnodes)[:-1]
    face_connectivity = np.split(nodes_flat0, cuts)
    print("[CASE] faces:", len(face_connectivity), "examples:", face_connectivity[:2])

    # (c) zone 정보(이름/타입) 예시
    zt = base["faces/zoneTopology"]
    zone_names = zt["name"][()].astype("S")  # 바이트 스트 -> 나중에 디코드
    # 어떤 파일은 가변 길이 문자열/정수 코드일 수 있음. 필요시 아래처럼 처리:
    try:
        zone_names = [n.decode("utf-8", "ignore") if isinstance(n, (bytes, np.bytes_)) else str(n)
                      for n in zone_names]
        print("[CASE] face zones (first 5):", zone_names[:5])
    except Exception as e:
        print("[CASE] zone names decode skipped:", e)

# ----------------------------
# 2) DATA: 결과 변수 자동 탐색/읽기
# ----------------------------
def list_datasets(g, prefix=""):
    out = []
    for k, v in g.items():
        p = f"{prefix}/{k}" if prefix else k
        if isinstance(v, h5py.Dataset):
            out.append(p)
        elif isinstance(v, h5py.Group):
            out.extend(list_datasets(v, p))
    return out

def try_get_first(ds_list, candidates):
    # 후보 이름 일부 일치(대소문자 무시)로 첫번째 발견 경로 반환
    low = [s.lower() for s in ds_list]
    for cand in candidates:
        for i, s in enumerate(low):
            if cand in s:
                return ds_list[i]
    return None

with h5py.File(data_path, "r") as f:
    # 모든 데이터셋 경로 나열
    if "results" not in f:
        print("[DATA] 'results' 그룹이 없습니다.")
    else:
        ds_all = list_datasets(f["results"], "results")
        print("[DATA] available datasets (first 20):", ds_all[:20])

        # 대표 스칼라/벡터 자동 탐색 (필요에 따라 후보 추가)
        press_path = try_get_first(ds_all, ["pressure", "static-pressure", "/p", "/press"])
        vel_path   = try_get_first(ds_all, ["velocity", "u-velocity", "v-velocity", "w-velocity", "uvw"])

        if press_path:
            p_vals = f[press_path][()]
            print(f"[DATA] pressure: {press_path}, shape={p_vals.shape}, sample={p_vals.ravel()[:5]}")
        else:
            print("[DATA] pressure-like dataset not found (add more candidates if needed).")

        if vel_path:
            v_vals = f[vel_path][()]
            # (N,3) 또는 (3,N) 정규화
            if v_vals.ndim == 2 and v_vals.shape[0] in (2,3):
                v_vals = v_vals.T
            print(f"[DATA] velocity:  {vel_path}, shape={v_vals.shape}, sample-row={v_vals[0] if v_vals.ndim==2 and len(v_vals)>0 else 'n/a'}")
        else:
            print("[DATA] velocity-like dataset not found.")


[CASE] coordinates: (10201, 2) (first 3 rows)
 [[ 0.00500001 -0.00500002]
 [ 0.00500002 -0.00510003]
 [ 0.00510002 -0.00500004]]
[CASE] faces: 20200 examples: [array([7125,  932], dtype=uint32), array([5060, 5061], dtype=uint32)]
[CASE] face zones (first 5): ['interior-fff_surface;wall;wall2;wall3;wall4']
[DATA] available datasets (first 20): ['results/1/phase-1/cells/SV_BF_V/1', 'results/1/phase-1/cells/SV_DENSITY/1', 'results/1/phase-1/cells/SV_LORENTZ_FORCE/1', 'results/1/phase-1/cells/SV_MU_LAM/1', 'results/1/phase-1/cells/SV_P/1', 'results/1/phase-1/cells/SV_U/1', 'results/1/phase-1/cells/SV_V/1', 'results/1/phase-1/cells/fields', 'results/1/phase-1/faces/SV_DENSITY/1', 'results/1/phase-1/faces/SV_FLUX/1', 'results/1/phase-1/faces/SV_P/1', 'results/1/phase-1/faces/SV_U/1', 'results/1/phase-1/faces/SV_V/1', 'results/1/phase-1/faces/SV_WALL_DIFFUSIVE_BC_MFLUX/1', 'results/1/phase-1/faces/SV_WALL_PRORUS_ZONE_FORCE_MEAN/1', 'results/1/phase-1/faces/SV_WALL_SHEAR/1', 'results/1/phase-1

In [ ]:
import h5py

# Fluent case 파일 열기
with h5py.File("C:/Users/spearlab05/Desktop/Galerkin ROM/Data/test.cas.h5", "r") as f:
    # 루트 그룹 확인
    print(list(f.keys()))

    # 예: Mesh 관련 데이터 접근
    if "Mesh" in f:
        print(list(f["Mesh"].keys()))

# Fluent data 파일 열기
with h5py.File("C:/Users/spearlab05/Desktop/Galerkin ROM/Data/test.dat.h5", "r") as f:
    # 루트 그룹 확인
    print(list(f.keys()))

    # 예: Solution 데이터 접근
    if "Results" in f:
        print(list(f["Results"].keys()))

['meshes', 'settings']
['results', 'settings']
